# Reactor models

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from majordome import (
    NormalFlowRate,
    toggle_warnings,
    toggle_reactor_warnings,
    composition_to_dict,
    composition_to_array,
    solution_report,
    copy_solution,
    copy_quantity,
)
from tabulate import tabulate
import cantera as ct

## Warnings

Module warnings can be controlled throught the following attributes:

In [3]:
help(toggle_reactor_warnings)

Help on function toggle_reactor_warnings in module majordome.reactor:

toggle_reactor_warnings(
    *,
    toggle_non_key_value: bool = True,
    toggle_missing_species_name: bool = True,
    toggle_unknown_species: bool = True,
    **kwargs
) -> None
    Reverse truth value of warning flags.



They can also be provided to the global `toggle_warnings`, which handles the equivalent function all `majordome` modules. Here we call it with no arguments so all warnings are toggled (and thus this example will be less polluted with warnings intentionaly promoted).

In [4]:
toggle_warnings()

## Utilities

Let's start by creating a standard Cantera solution:

In [5]:
solution = ct.Solution("airish.yaml")
solution.TPY = 273.15, ct.one_atm, "N2: 0.78 O2: 0.21, AR: 0.01"

Conversion of composition strings with name filtering is available:

In [6]:
composition_to_dict("O2: 1, teste: 1", solution.species_names)

{'O2': 1.0}

It is also possible to set unit composition with species names; if you do not provide a validation list, all are are kept:

In [7]:
composition_to_dict("O2, N2, hello")

{'O2': 1.0, 'N2': 1.0, 'hello': 1.0}

When working with arrays, take care not to end up in the following situation:

In [8]:
composition_to_array(", teste: 1", solution.species_names)

array([0., 0., 0.])

There is also a helper for generating inputs for use with `tabulate.tabulate`:

In [9]:
data = solution_report(solution, specific_props=True,
                       composition_spec="mass", selected_species=[])
print(tabulate(data))

----------------------  --------  ------------
Temperature             K            273.15
Pressure                Pa        101325
Density                 kg/m³          1.28735
Specific enthalpy       J/(kg.K)  -25114.9
Specific heat capacity  J/(kg.K)    1004.95
mass: AR                -              0.01
mass: N2                -              0.78
mass: O2                -              0.21
----------------------  --------  ------------


Because sometimes Cantera lacks hard-copy utilities for certain classes, we provide simple wrappers that create new instances and set the state to the same of the source object. Nothing checked, nothing tested. A first of this kind is `copy_solution`, illustrated below:

In [10]:
newairs = copy_solution(solution)
newairs.TPX = 373.15, None, newairs.X
print(tabulate(solution_report(newairs)))

----------------------  --------  -------------
Temperature             K            373.15
Pressure                Pa        101325
Density                 kg/m³          0.942356
Specific enthalpy       J/(kg.K)   75902.1
Specific heat capacity  J/(kg.K)    1015.82
mass: AR                -              0.01
mass: N2                -              0.78
mass: O2                -              0.21
----------------------  --------  -------------


In [11]:
print(tabulate(solution_report(solution)))

----------------------  --------  ------------
Temperature             K            273.15
Pressure                Pa        101325
Density                 kg/m³          1.28735
Specific enthalpy       J/(kg.K)  -25114.9
Specific heat capacity  J/(kg.K)    1004.95
mass: AR                -              0.01
mass: N2                -              0.78
mass: O2                -              0.21
----------------------  --------  ------------


In addition to this, there is `copy_quantity`, which proves quite useful in establishing an *algebra* of mixtures.

In [12]:
air = copy_solution(solution)

air1 = ct.Quantity(air, mass=1.0)
air1.TPX = 373.15, None, newairs.X

air2 = copy_quantity(air1)
air2.TPX = 273.15, None, air2.X

mixair = air1 + air2
air1.T, air2.T, mixair.T

(373.15, 273.15, 323.337485365699)

## *NormalFlowRate*

Common daily work activity for the process engineer is to perform mass balances, but wait, ..., gas flow rates are generally provided under normal conditions, and compositions may vary, so you need to compute normal densities first... whatever. This class provides a calculator wrapping a Cantera solution object so that your life gets easier.

Its simples use case is as follows:

In [13]:
nfr = NormalFlowRate("airish.yaml")
print(f"Convert 1000 Nm³/h to {nfr(1000.0):.5f} kg/s")

Convert 1000 Nm³/h to 0.35903 kg/s


If the database file default composition does not suit you, no problems:

In [14]:
nfr = NormalFlowRate("airish.yaml", X="N2: 1")
print(f"Convert 1000 Nm³/h to {nfr(1000.0):.5f} kg/s")

Convert 1000 Nm³/h to 0.34718 kg/s


You can also print a nice report to inspect the details of internal state. For more, please check its documentation at the [API page](../modules/reactor.md).

In [15]:
print(nfr.report())

|------------------------|----------|--------------|
| Temperature            | K        |    273.15    |
| Pressure               | Pa       | 101325       |
| Density                | kg/m³    |      1.24985 |
| Specific enthalpy      | J/(kg.K) | -25864.9     |
| Specific heat capacity | J/(kg.K) |   1035.52    |
| mass: N2               | -        |      1       |
